# **Data Visualization Notebook**

## Objectives

- Address business requiremenent 1:
    - The client is interested in conducting a study to visually differentiate a cherry leaf that is healthy from one that contains powdery mildew.

## Inputs

This notebook will use the following inputs:
- inputs\cherry-leaves\cherry-leaves\test
- inputs\cherry-leaves\cherry-leaves\train
- inputs\cherry-leaves\cherry-leaves\validation

## Outputs

This notebook will generate the following outputs:
- A pickle file storing image shape embeddings (image data stored as a numercal representation) 
- Plots that show the mean (average) image and variability of images for each label
- A plot to show the difference beween healthy leaves and mildewed leaves
- Code to address business requirement 1 and build an image montage

## Set the Data Visualisazation Directories

### Import the required libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
sns.set_style("white")
from matplotlib.image import imread

### Set the working directory

Identify the current working directory

In [2]:
current_dir = os.getcwd()
current_dir

'/workspaces/milestone-project-mildew-detection-in-cherry-leaves/jupyter_notebooks'

Set a new working directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Check the new working directory

In [4]:
current_dir = os.getcwd()
current_dir

'/workspaces/milestone-project-mildew-detection-in-cherry-leaves'

### Set the input directories

Establish the train, test and validation directory paths

In [5]:
image_data = 'inputs/cherry-leaves/cherry-leaves'
train_path = image_data + '/train'
val_path = image_data + '/validation'
test_path = image_data + '/test'

Set the ouptut directory

In [6]:
# Define the version of the output directory
version = 'v1'
current_dir = os.getcwd()  # Get the current working directory

# Construct the full path for the versioned output folder
file_path = os.path.join(current_dir, 'outputs', version)

# Check if the directory for this version already exists
if os.path.exists(file_path):
    print(f'Old version "{version}" already exists. Create a new version.')
else:
    try:
        # Create the directory if it doesn't exist, `exist_ok=True` prevents errors if it already exists
        os.makedirs(file_path, exist_ok=True)
        print(f'Created new directory: {file_path}')
    except OSError as e:
        # Print an error message if directory creation fails
        print(f'Error creating directory: {e}')

Created new directory: /workspaces/milestone-project-mildew-detection-in-cherry-leaves/outputs/v1


### Set the label names

Labels should be 'healthy' and 'powdery_mildew'.

In [7]:
# Set the labels
labels = os.listdir(train_path)
print('The labels for the images are', labels)

The labels for the images are ['healthy', 'powdery_mildew']
